In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from indexing.processor import IndexingProcessor
from indexing.processor import IndexingProcessor
from search.qdrant import QdrantSearchEngine
from search.bm25 import ElasticsearchBM25
import json
from search.hybrid_search import search

In [11]:

qdrant_engine = QdrantSearchEngine()
es_engine =  ElasticsearchBM25()
await es_engine.initialize()


processor = IndexingProcessor(qdrant_engine=qdrant_engine, es_engine=es_engine)

Connecting to Elasticsearch at localhost:9200
Index meetings already exists


In [12]:
with open('data/meeting.json', 'r') as f:
    data = json.load(f)

print(data)

{'id': 3863, 'platform': 'google_meet', 'native_meeting_id': 'zwm-isdv-yvb', 'constructed_meeting_url': 'https://meet.google.com/zwm-isdv-yvb', 'status': 'completed', 'start_time': '2025-07-21T09:31:26.273071', 'end_time': '2025-07-21T10:21:43.104087', 'segments': [{'start': 0.0, 'end': 4.72, 'text': ' in my college itself, but it was a corporate hackathon.', 'language': 'en', 'created_at': '2025-07-21T09:35:27.944067', 'speaker': 'Garbhit Sharma', 'absolute_start_time': '2025-07-21T09:34:09.039974+00:00', 'absolute_end_time': '2025-07-21T09:34:13.759974+00:00'}, {'start': 4.72, 'end': 10.28, 'text': ' I participated in core cybersecurity domain and fortunately I was the winner among the', 'language': 'en', 'created_at': '2025-07-21T09:35:27.944282', 'speaker': 'Garbhit Sharma', 'absolute_start_time': '2025-07-21T09:34:13.759974+00:00', 'absolute_end_time': '2025-07-21T09:34:19.319974+00:00'}, {'start': 10.28, 'end': 16.92, 'text': ' corporate teams which have experienced more than 5 y

In [13]:
df =pd.DataFrame(data['segments'])

In [16]:
speakers = df['speaker'].dropna().unique().tolist()
start_datetime = pd.to_datetime(df['absolute_start_time'].min())
content_id = '1'

In [ ]:
r = await processor._merge_chunks(df, content_id, start_datetime, speakers)
await processor._index_to_search_engines(r[0], r[1])

The chunk is part of a conversation between Garbhit Sharma and Dmitry Grankin, where Garbhit discusses his experiences in hackathons, particularly highlighting his success in a corporate hackathon focused on cybersecurity and his excitement about a new project he is working on.
This chunk occurs towards the end of a conversation between Dmitry Grankin and Garbhit Sharma, where they discuss collaboration on AI projects, the potential for learning and growth, and express mutual excitement about their respective work in hackathons and open source contributions.
This chunk is part of a conversation between Garbhit Sharma and Dmitry Grankin, where Garbhit discusses his motivations for contributing to open source projects, the networking opportunities it provides, and how it relates to his participation in hackathons, particularly mentioning a recent Web3 hackathon.
This chunk is part of a conversation between Garbhit Sharma and Dmitry Grankin, where they discuss machine learning projects, s

In [17]:
await search('what is the most important topic of the meeting?',qdrant_engine=qdrant_engine, es_engine=es_engine)

NameError: name 'search' is not defined